# Un Hello WebAssembly écrit en Rust

---

> Dans ce chapitre, vous allez créer votre premier fichier wasm à partir de code écrit en Rust puis l'exécuter.

## Création d'un nouveau projet Rust

Tout commence en Rust par la magie de Cargo, un projet destine au web n'échappe pas à cette règle. Ainsi la première commande sera un **cargo new**.

- Rendez-vous sur l'image Docker grâce à la commande `docker compose exec boilerplate bash` ou en exécutant le script `shell-docker.sh` puis exécutez cargo :

`cargo new --lib first-step`

> Contrairement à ce que nous avons vu dans le module principal Rust, il ne sera pas question ici de faire un fichier exécutable à partir d'une fonction main, mais une librairie qui sera utilisée en tant que tel par le navigateur web. Donc au lieu de nous avoir généré un fichier main.rs, la commande ci-dessus avec le `--lib` a écrit un fichier `lib.rs`.

```
.
 |-Cargo.toml
 |-.git
 | |-info
 | ... Some git stuff ...
 |-.gitignore
 |-src
 | |-lib.rs

```

## Configuration de cargo

Puisque nous allons utiliser la crate wasm-bindgen afin de pouvoir interfacer le code Rust avec JavaScript, il est nécessaire de l'ajouter aux dépendances.

La communauté Rust écrit une incroyable multitude de crates qui se trouvent sur [crates.io](crates.io), c'est là que nous trouverons la crate wasm-bindgen. *Souvenez-vous qu'une crate représente une unité de compilation.*

**Lançons la recherche wasm-bindgen :**

![CRATES_IO_BINDGEN](pictures/wasm_bindgen_crates.io.png)

*Vous pouvez constater que l'équipe de dev de wasm-bindgen donne un exemple complet sur le site crates.io, ce sera souvent le cas de toutes les crates que vous utiliserez.*

A droite de la page, au mileu vous avezla ligne qu'il vous faudrait copier dans la partie `[dependencies]` de votre fichier Cargo.toml :

> `wasm-bindgen = "0.2.86"` 

Enfin, on rajoutera cette section au fichier :

```
[lib]
crate-type = ["cdylib"]
```

*cdylib signifie librairie dynamique, c'est le seul style de crate qui fonctionne avec wasm-pack...*

**Le fichier `Cargo.toml` sera donc ainsi :**

```
[package]
name = "first-step"
version = "0.1.0"
edition = "2021"

# See more keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html

[lib]
crate-type = ["cdylib"]

[dependencies]
wasm-bindgen = "0.2.86"
```

## Le code minimal

> Pour ce qui est de notre fichier lib.rs du dossier .src/, il est peut-être avisé de copier l'exemple donné par la team de wasm-bindgen sur crates.io. Les exemples proposés par les crates sont souvent ainsi utilisés :
> - On les copie.
> - On regarde si ça fonctionne et comment.
> - On consulte la documentation de la crate pour en savoir plus sur ses fonctionnalités.
> - Et on enrichit l'utilisation que l'on en fait.

- Remplaçons le contenu du fichier `lib.rs` par cet exemple :

```
use wasm_bindgen::prelude::*;

// Import the `window.alert` function from the Web.
#[wasm_bindgen]
extern "C" {
    fn alert(s: &str);
}

// Export a `greet` function from Rust to JavaScript, that alerts a
// hello message.
#[wasm_bindgen]
pub fn greet(name: &str) {
    alert(&format!("Hello, {}!", name));
}
```

### Importation du prélude de wasm-bindgen

`use wasm_bindgen::prelude::*;`

Nous avons déjà vu le `use` dans la partie sur les namespaces du chapitre 002 de la formation Rust, on importe avec le wildcard `*` tout ce qui est présent dans le namespace wasm_bindgen::prelude.

`cargo doc --open`

![BIND_PRELUDE](pictures/wasm_prelude.png)

**Ici ce qui va nous servir c'est la macro procédurale `wasm_bindgen`.**

**Cette macro fait la conversion entre les types Rust et les types Javascript de façon transparente.**

*Pour information, les macros procédurales permettent de créer des extensions de syntaxe, mais ce n'est pas la question dans cette formation*.



### La déclaration d'une fonction externe

Lorsque l'on utilise Rust à des fins de FFI afin de pouvoir communiquer avec d'autres langages de programmation, les fonctions étrangères au rust à appeler seront notées comme `extern`, elles ne sont donc pas définies dans la crate. *Dans un programme C + Rust, c'est le linker qui fera les correspondances d'adresse entre les fonctions.*

> Ici, on indique à la librairie qu'une fonction externe nommée `alert` existe et qu'elle est appelable en lui passant une chaine de caractère en paramètre.

### Création d'une fonction à appeler depuis Javascript

Enfin vient la définition d'une fonction nommée `greet` qui sera appelable depuis le code javascript avec une chaine de caractère en paramètre. Là aussi la macro `#[wasm_bindgen]` doit être utilisée, et d'une façon générale, on utilisera cette macro pour toutes les définitions de fonction à appeler et qui sont appelées. Les fonctions internes a notre code Rust n'auront pas besoin de cette macro.

Dans la fonction `greet`, on appelle la fonction alert avec une chaine de caractère en paramètre. Notez que la macro `format!` est d'abord utilisé pour générer un type `String` et la référence de cette dernière sera une `&str`.

## Compilation de notre exemple en web assembly

L'étape suivante est bien entendu de générer le fichier .wasm, pour faire cela, nous allons directement utiliser le programme `wasm-pack` qui se chargera à la fois de compiler le code Rust en assembleur web en utilisant la target `wasm32-unknown-unknown` et a la fois de générer la glu javascript.

`wasm-pack build --target web`

- un package resultant sera creer dans un dossier .pkg/ a la racine du projet :

```
 | ... Some git stuff ...
 | | |-pack
 | | |-info
 |-pkg
 | |-first_step_bg.wasm
 | |-.gitignore
 | |-first_step.js
 | |-first_step_bg.wasm.d.ts
 | |-first_step.d.ts
 | |-package.json
 |-.gitignore
 | ... Other stuff ...
```

**ATTENTION : LES NOMS DES FICHIERS GÉNÉRÉS ONT CHANGE, LE SYMBOLE MINUS `-` EST DEVENU UN UNDERSCORE `_`.**

## Le fichier index.html

> Enfin, il manque le plus important, le fichier html qui lancera notre code assembleur, voici un exemple très simple :

```
<!DOCTYPE html>
<html lang="en-US">
    <head>
        <meta charset="utf-8" />
            <title>hello-wasm example</title>
    </head>
    <body>
        <script type="module">
            import init, { greet } from "./pkg/first_step.js";
            init().then(() => {
                greet("WebAssembly");
            });
        </script>
    </body>
</html>
```
**Ce fichier devra être placé à la racine du projet.**

## Le lancement

> Afin de pouvoir visualiser cet hello web assembly, vous pouvez utiliser le programme simple-http-server disponible sur l'image docker. À la racine du projet, écrivez :

`simple-http-server -p 8082 ./`

**Ensuite, dans votre navigateur sur l'hôte, vous pouvez vous rendre à l'adresse `127.0.0.1:8082` :**
![TREE](pictures/tree.png)

**Sélectionnez enfin index.html :**
![ALERT](pictures/alert.png)

**Félicitations, vous venez de réaliser votre premier Hello World en WebAssembly avec le langage de programmation Rust ! Dans les chapitres suivants, nous verrons comment ajouter de nouvelles fonctionnalités à ce programme web.**

*Grâce à la console web dev, nous pouvons constater que le fichier .wasm a été téléchargé depuis le serveur.*

![WASM](pictures/fichier_wasm.png)